# Tutorial for data analysis using PySpark

This tutorial is based on the LinkedIn Learning example "Apache PySpark by Example" by Jonathan Fernandes.

The first steps needed to run this tutorial comprise installing PySpark, creating a Spark Session and downloading the data (to the virtual environment).

### Install PySpark and create Spark session

In [ ]:
!pip install pyspark==3.5.1

Create new Spark session.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

### Download the City of Chicago's Reported Crime Dataset:

In [ ]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!ls -l

In [ ]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
!ls -l

Create PySpark's dataframe object with data from the csv file and print 5 rows.

In [ ]:
from pyspark.sql.functions import to_timestamp, col, lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

## Working with Schemas

The next lines of code illustrate how one to define the schema of a dataframe.

Check automatically created schema for the dataset (i.e., column names and types)

In [ ]:
rc.printSchema()

It's usually better to define the schema so that Spark can efficiently work on the data without having to infer the types of variables in the data.

Let's import some functions to define the schema.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

Using the column names, we can create a schema variable in which each column has a specific data type.

In [ ]:
schema = StructType([
    StructField('ID', StringType(), True),
 StructField('Case Number', StringType(), True),
 StructField('Date', TimestampType(), True),
 StructField('Block', StringType(), True),
 StructField('IUCR', StringType(), True),
 StructField('Primary Type', StringType(), True),
 StructField('Description', StringType(), True),
 StructField('Location Description', StringType(), True),
 StructField('Arrest', StringType(), True),
 StructField('Domestic', BooleanType(), True),
 StructField('Beat', StringType(), True),
 StructField('District', StringType(), True),
 StructField('Ward', StringType(), True),
 StructField('Community Area', StringType(), True),
 StructField('FBI Code', StringType(), True),
 StructField('X Coordinate', StringType(), True),
 StructField('Y Coordinate', StringType(), True),
 StructField('Year', IntegerType(), True),
 StructField('Updated On', StringType(), True),
 StructField('Latitude', DoubleType(), True),
 StructField('Longitude', DoubleType(), True),
 StructField('Location', StringType(), True)
])

In alternative, we can create a list if tuples. Each tuple contains the column name and variable type.

In [ ]:
labels = [
    ('ID', StringType()),
 ('Case Number', StringType()),
 ('Date', TimestampType()),
 ('Block', StringType()),
 ('IUCR', StringType()),
 ('Primary Type', StringType()),
 ('Description', StringType()),
 ('Location Description', StringType()),
 ('Arrest', StringType()),
 ('Domestic', BooleanType()),
 ('Beat', StringType()),
 ('District', StringType()),
 ('Ward', StringType()),
 ('Community Area', StringType()),
 ('FBI Code', StringType()),
 ('X Coordinate', StringType()),
 ('Y Coordinate', StringType()),
 ('Year', IntegerType()),
 ('Updated On', StringType()),
 ('Latitude', DoubleType()),
 ('Longitude', DoubleType()),
 ('Location', StringType())
 ]

The use a list comprehension to create the schema variable:

In [ ]:
schema = StructType([StructField (x[0], x[1], True) for x in labels])

Read csv data into a Data Frame now using the schema for efficiency. Print schema to check if everything looks as expected. If for some reason the schema doesn't work, use the automatically assigned one.

In [ ]:
rc = spark.read.csv('reported-crimes.csv', schema=schema)
rc.printSchema()

## Working with Columns

The next lines of code illustrate how one can use PySpark to manipulate the columns of a dataset.

### Display only the first 5 rows of the column name IUCR

Show the first rows of a particular column (e.g., the IUCR column)

In [ ]:
rc.select('IUCR').show(5)

A different syntax to do the same thing.

In [ ]:
rc.select(rc.IUCR).show(5)

A different syntax to do the same thing.

In [ ]:
rc.select(col('IUCR')).show(5)

### Display only the first 4 rows of the column names Case Number, Date and Arrest

In [ ]:
rc.select('Case Number', 'Date', 'Arrest').show(5)

### Add a new column with the value 1 for all rows and the column name 'One'.

In [ ]:
rc.withColumn('One', lit(1)).show(5)

### Drop the column IUCR and show the data frame's first 5 rows.

In [ ]:
rc = rc.drop('IUCR')
rc.show(5)

## Working with Rows

The next lines of code illustrate how one can use PySpark to manipulate the rows of a dataset.

### Add the reported crimes for an additional day, 12-Nov-2018, to our dataset.

Create a new data frame from the reported crime dataset and create an additional data frame (called one_day) with one row only for the reported crime on the 12th of November of 2018.

In [ ]:
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
one_day = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-12'))

Confirm that the additional data frame only has one row.

In [ ]:
one_day.count()

Now use union to add the additional row to the original data frame.

In [ ]:
rc.union(one_day).show(5)

Confirm that the new row has been added.

In [ ]:
rc.union(one_day).orderBy('Date', ascending=False).show(5)

### What are the top 10 number of reported crimes by Primary type, in descending order of occurence?

Now let's check the number of crimes reported by type:

In [ ]:
rc.groupBy('Primary Type').count().show()

And reorder them according from the most reported to the least reported showing the first 10 rows:

In [ ]:
rc.groupBy('Primary Type').count().orderBy('count', ascending=False).show(10)

## Challenge

- What is the percentage of reported crimes that resulted in an arrest?
- What are the top 3 locations for reported crimes?

To answer the first question, let's reimport the data.

In [ ]:
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))

### What percentage of reported crimes resulted in an arrest?

Now let's check how many unique variables we have in the column 'Arrest':

In [ ]:
rc.select('Arrest').distinct().show()

In order to select the correct Arrest variables, let's first check their data type:

In [ ]:
rc.printSchema()

Since 'Arrest' variables are strings we can run the following line of code to select how many crimes lead to an arrest, dividing by the total number of crimes (or rows in the dataset). This provides the percentage of reported crimes that resulted in an arrest.

In [ ]:
rc.filter(col('Arrest') == 'true').count() / rc.select('Arrest').count()

### What are the top 3 locations for reported crimes?

For the second question, we can group the data by location, as follows:

In [ ]:
rc.groupBy('Location Description').count().show()

Then count how many rows we have for each location and order in a descending order to find out the top locations for reported crime:

In [ ]:
rc.groupBy('Location Description').count().orderBy('count', ascending=False).show(3)

## Working with Functions

The next lines of code illustrate how one can use PySpark's built-in functions to manipulate data.

Import all functions to see the list of built-in functions available.

In [ ]:
from pyspark.sql import functions

Display full list:

In [ ]:
print(dir(functions))

### Display the Primary Type column in lower and upper characters, and the first 4 characters of the column

Load useful functions.

In [ ]:
from pyspark.sql.functions import lower, upper, substring

In [ ]:
rc.select(lower(col('Primary Type')), upper(col('Primary Type')), substring(col('Primary Type'),1,4)).show(5)

### Show the oldest date and the most recent date

Load min max functions to get these dates.

In [ ]:
from pyspark.sql.functions import min, max

In [ ]:
rc.select(min(col('Date')), max(col('Date'))).show(1)

### What is 3 days earlier that the oldest date and 3 days later than the most recent date?

In [ ]:
from pyspark.sql.functions import date_add, date_sub

In [ ]:
rc.select(date_sub(min(col('Date')),3), date_add(max(col('Date')),3)).show(1)

## Working with Dates

The next lines of code illustrate how one can work with different formats for dates.
For a full list of formats check here:
https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

Import useful functions for formatting dates.

In [ ]:
from pyspark.sql.functions import to_date, to_timestamp

Create new dataframe with a single row for the 25th of December of 2019 at 13h30. This dataframe will be used to convert the date into different formats.

### Convert to 2019-12-25 13:30:00

In [ ]:
df = spark.createDataFrame([('2019-12-25 13:30:00',)], ['Christmas'])
df.show(1)

In [ ]:
df.select(to_date(col('Christmas'),'yyyy-MM-dd HH:mm:ss'), to_timestamp(col('Christmas'),'yyyy-MM-dd HH:mm:ss')).show(1)

### Convert to 25/Dec/2019 13:30:00

In [ ]:
df = spark.createDataFrame([('25/Dec/2019 13:30:00',)], ['Christmas'])
df.show(1)

In [ ]:
df.select(to_date(col('Christmas'),'dd/MMM/yyyy HH:mm:ss'), to_timestamp(col('Christmas'),'dd/MMM/yyyy HH:mm:ss')).show(1)

### Convert to 12/25/2019 01:30:00 PM

In [ ]:
df = spark.createDataFrame([('12/25/2019 01:30:00 PM',)], ['Christmas'])
df.show(1, truncate=False)

In [ ]:
df.select(to_date(col('Christmas'),'MM/dd/yyyy hh:mm:ss a'), to_timestamp(col('Christmas'),'MM/dd/yyyy hh:mm:ss a')).show(1)

## Working with Joins

The next lines of code illustrate how to join different datasets by showing how to join the reported crimes dataset with a new dataset from the city of Chicago with information on police stations. The datasets will be joined by the district names.

Start by reimporting the data.

In [ ]:
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))

### Download police station data

In [ ]:
!wget -O police_station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
!ls -l

Create a new dataframe for the police station dataset and show first 5 rows.

In [ ]:
ps = spark.read.csv('police_station.csv', header=True)
ps.show(5)

To make things run faster one can cache the entire dataset for this example:

In [ ]:
rc.cache()
rc.count()

Check how the district name is stored in the reported crime and police stations datasets.

In [ ]:
ps.select(col('District')).distinct().show(30)

In [ ]:
rc.select('District').distinct().show(30)

It's clear that the police stations data needs to be left padded to match the same district names in the reported crime dataframe.

In [ ]:
from pyspark.sql.functions import lpad

In [ ]:
ps.select(lpad(col('District'),3,'0')).show()

Add new column with the new formatted district names (left padded) to the police stations dataframe.

In [ ]:
ps = ps.withColumn('Format_District', lpad(col('District'),3,'0'))
ps.show(5)

### The reported crimes dataset has only the district number. Add the district name by joining with the police station dataset

In [ ]:
rc.join(ps, rc.District == ps.Format_District, 'left_outer').show()

## Challenge

- What is the most frequently reported non-criminal activity?
- Using a bar chart, plot which day of the week has the most number of reported crime.

### What is the most frequently reported non-criminal activity?

Count all the unique types of reported crimes to check for non criminal activities.

In [ ]:
rc.select(col('Primary Type')).distinct().count()

Order the results to see better.

In [ ]:
rc.select(col('Primary Type')).distinct().orderBy(col('Primary Type')).show(36, truncate=False)

Now select all the non-criminal activities.

In [ ]:
nc = rc.filter((col('Primary Type')=='NON - CRIMINAL') | (col('Primary Type')=='NON-CRIMINAL') | (col('Primary Type')=='NON-CRIMINAL (SUBJECT SPECIFIED)'))

In [ ]:
nc.show(50)

And order them from most to least frequent.

In [ ]:
nc.groupBy(col('Description')).count().orderBy('count', ascending=False).show(truncate=False)

### Using a bar chart, plot which day of the week has the most number of reported crime.

Import dayofweek function.

In [ ]:
from pyspark.sql.functions import dayofweek

Check if this function returns the desired output (the day of the week).

In [ ]:
rc.select(col('Date'), dayofweek(col('Date'))).show(5)

It returns a number, so it's not ideal. We need to convert it into the actual day.

In [ ]:
from pyspark.sql.functions import date_format

Check if the new function returns the correct day of the week.

In [ ]:
rc.select(col('Date'), dayofweek(col('Date')), date_format(col('Date'), 'E' )).show(5)

Group all the reported crimes using the day of the week, count the number of crimes for each day, and order in descending order. The day of the week with the most number of reported crimes is Friday:

In [ ]:
rc.groupBy(date_format(col('Date'), 'E')).count().orderBy('count', ascending=False).show()

Now, to make the plot, collect all these data.

In [ ]:
rc.groupBy(date_format(col('Date'), 'E')).count().collect()

And create two lists, one with the day of the week string, and one with the number of reported crimes.

In [ ]:
dow = [x[0] for x in rc.groupBy(date_format(col('Date'), 'E')).count().collect()]
dow

In [ ]:
cnt = [x[1] for x in rc.groupBy(date_format(col('Date'), 'E')).count().collect()]
cnt

Now let's import pandas and matplotlib for plotting. First we create a Pandas dataframe with the above lists, using a dictionary as shown below:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

cp = pd.DataFrame({'Day_of_week' : dow, 'Count' : cnt})
cp

We can then use the plotting functionality from Pandas and matplotlib to create a bar plot:

In [ ]:
cp.sort_values('Count', ascending=False).plot(kind='bar', x='Day_of_week', y='Count', color='olive')
plt.xlabel('Day of the week')
plt.ylabel('No. of reported crimes')
plt.title('No. of reported crimes per day of the week from 2001 to present')